# Accountablity

Numbers of cases by discipline and grand jury. Relates to story [Texas police rarely faulted as 289 suspects died in their custody](http://www.mystatesman.com/news/statesman-investigates-texas-police-rarely-faulted-289-suspects-died-their-custody/evCD6YyzMSZQi4c06QKfJP/)

In [1]:
import agate

In [2]:
# sets data types on fields agate got wrong
specified_data_types = {
    'tracked_cause': agate.Text(),
    'offense': agate.Text(),
    'case_study': agate.Text(),
    'official_discipline': agate.Text()
}
deaths = agate.Table.from_csv('../exports/deaths_latest.csv', column_types=specified_data_types)

print(deaths)

# get and print total number of cases
deaths_total_count = len(deaths)
print('Total deaths: {}\n'.format(deaths_total_count))


| column               | data_type |
| -------------------- | --------- |
| id                   | Number    |
| ag_report_url        | Text      |
| first_name           | Text      |
| middle_name          | Text      |
| last_name            | Text      |
| suffix               | Text      |
| slug                 | Text      |
| race                 | Text      |
| gender               | Text      |
| date_of_birth        | Date      |
| date_of_death        | Date      |
| age                  | Number    |
| agency               | Number    |
| restrained           | Boolean   |
| tazed                | Boolean   |
| times_tazed          | Number    |
| pepper_sprayed       | Boolean   |
| official_discipline  | Text      |
| grand_jury_result    | Text      |
| mental_health_issues | Boolean   |
| manner_of_death      | Text      |
| drug_intoxication    | Boolean   |
| cause_of_death       | Text      |
| tracked_cause        | Text      |
| offense              | Text      |
|

In [3]:
deaths_grand_jury = deaths.pivot('grand_jury_name')

print('Breaks down grand jury, out of {} total cases:\n'.format(
    deaths_total_count))
deaths_grand_jury.print_table()

Breaks down grand jury, out of 289 total cases:

| grand_jury_name | Count |
| --------------- | ----- |
|                 |   129 |
| Not brought     |    82 |
| No-bill         |    76 |
| Indictment      |     2 |


In [4]:
deaths_discipline = deaths.pivot('discipline_name')

print('Breaks down discipline, out of {} total cases:\n'.format(
    deaths_total_count))
deaths_discipline.print_table()

Breaks down discipline, out of 289 total cases:

| discipline_name | Count |
| --------------- | ----- |
|                 |   286 |
| Fired           |     3 |


In [5]:
# sets data types on fields agate got wrong
specified_data_types = {
    'tracked_cause': agate.Text(),
    'offense': agate.Text(),
    'case_study': agate.Text(),
    'official_discipline': agate.Text()
}
deaths_with_notes = agate.Table.from_csv(
#     '../data_raw/Death-2017-06-13.csv',
    '../exports/deaths_latest.csv',
    column_types=specified_data_types
)

# get and print total number of cases
print('Total deaths: {}\n'.format(
    len(deaths_with_notes)
))


Total deaths: 289



### Settlements

When reporters found there was a settlement in a case, they added it to the notes field in the database. This query finds those cases. By [Dan Keemahill](dankeemahill).

In [6]:
import re

deaths_with_notes.where(
    # notes field contains "$" or "settl"
    # this search was discussed during iTeam meeting
    lambda row: re.search(r'(\$)|settl',
                          str(row['notes']))
    ).select(
    ['id',
     'first_name',
     'middle_name',
     'last_name'
    ]).print_table()

|  id | first_name | middle_name | last_name |
| --- | ---------- | ----------- | --------- |
| 667 | Michael    | Louis       | Peden     |
| 637 | Joel       | Don         | Casey     |
| 550 | Corey      | Dean        | Bailey    |
| 497 | Kevin      | Jibreel     | Laday     |
| 491 | Jamaal     | Ray         | Valentine |
| 450 | Michael    | Patrick     | Jacobs    |


In [7]:
# use regex to find settle amounts for Robert
payload = []

for row in deaths_with_notes.where(
    lambda row: re.search(r'(\$)|settl',
                          str(row['notes'])) or
        re.search(r'(\$)|settl',
                 str(row['investigation']))
    # iterate settlement cases
    ).rows:
    if re.search(r'\$\d.*', str(row['notes'])):
        dollar_amt = (re.search(r'\$\d.*', str(row['notes']))
                       .group())
        
        payload.append({
            'name': '{} {} {}'.format(
                row['first_name'],
                row['middle_name'],
                row['last_name']
            ),
            'id': row['id'],
            'dollar_amt': ' '.join(dollar_amt.replace('Settlement', '')
                                             .split(u' ')[:2])
        })
        
    if re.search(r'\$\d.*', str(row['investigation'])):
        dollar_amt = (re.search(r'\$\d.*', str(row['investigation']))
                       .group())
        
        payload.append({
            'name': '{} {} {}'.format(
                row['first_name'],
                row['middle_name'],
                row['last_name']
            ),
            'id': row['id'],
            'dollar_amt': ' '.join(dollar_amt.replace('Settlement', '')
                                             .split(u' ')[:2])
        })
        
# display
agate.Table.from_object(
    payload
).print_table()

| dollar_amt    | name                 |  id |
| ------------- | -------------------- | --- |
| $3 million    | Joel Don Casey       | 637 |
| $1.25 million | Jonathan Ryan Paul   | 619 |
| $800,000      | Corey Dean Bailey    | 550 |
| $49,000       | Juan Manuel Nunez    | 496 |
| $2 million    | Michael Patrick J... | 450 |
